In [3]:
pwd

'/Users/aiden/Dropbox/Mac/Desktop/dash_plotly/corona_kaggle'

In [35]:
# essential libraries
import os
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# for offline ploting
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
import plotly.io as pio
pio.renderers.default = "notebook_connected"

# 데이터셋
time = pd.read_csv('data/Time.csv')
timeAge = pd.read_csv('data/TimeAge.csv')
timeGender = pd.read_csv('data/TimeGender.csv')
timeProvince = pd.read_csv('data/TimeProvince.csv')

# color pallette
cnf, dth, rec, act = '#393e46', '#ff2e63', '#21bf73', '#fe9801' 
DEFAULT_PLOTLY_COLORS=['rgb(31, 119, 180)', 'rgb(255, 127, 14)',
                       'rgb(44, 160, 44)', 'rgb(214, 39, 40)',
                       'rgb(148, 103, 189)', 'rgb(140, 86, 75)',
                       'rgb(227, 119, 194)', 'rgb(127, 127, 127)',
                       'rgb(188, 189, 34)', 'rgb(23, 190, 207)']

# 기본 세팅 설정(font 등)
layout_setting = {'font':dict(size=18, color='#60606e', family='Franklin Gothic' )}

In [9]:
time

,date,time,test,negative,confirmed,released,deceased
0,2020-01-20,16,1,0,1,0,0
1,2020-01-21,16,1,0,1,0,0
2,2020-01-22,16,4,3,1,0,0
3,2020-01-23,16,22,21,1,0,0
4,2020-01-24,16,27,25,2,0,0
...,...,...,...,...,...,...,...
158,2020-06-26,0,1232315,1200885,12602,11172,282
159,2020-06-27,0,1243780,1211261,12653,11317,282
160,2020-06-28,0,1251695,1219975,12715,11364,282
161,2020-06-29,0,1259954,1228698,12757,11429,282


# Scatter Plot : 여러 그래프 겹쳐 그리기

In [17]:
time.head(3)

,date,time,test,negative,confirmed,released,deceased
0,2020-01-20,16,1,0,1,0,0
1,2020-01-21,16,1,0,1,0,0
2,2020-01-22,16,4,3,1,0,0


In [36]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=time['date'],y=time['released'],  mode='lines+markers', name='released'))
fig.add_trace(go.Scatter(x=time['date'],y=time['confirmed'], mode='lines+markers', name='confirmed'))
fig.add_trace(go.Scatter(x=time['date'],y=time['deceased'],  mode='lines+markers', name='deceased'))

fig.update_layout(title='<b>시간에 따른 확진자 추이</b>', **layout_setting)

fig.show()

# Bar 그래프 : 그래프 색깔 지정

In [37]:
fig = go.Figure()
fig.add_trace(go.Bar(x=time['date'],y=time['confirmed'].diff(), name='confirmed', marker_color='rgba(152, 0, 0, .8)'))

fig.update_layout(title='<b>일단위 확진자 수</b>', **layout_setting)
fig.show()

# Bar 그래프 : 특정 부분만 색깔 주기

In [18]:
time.head(3)

,date,time,test,negative,confirmed,released,deceased
0,2020-01-20,16,1,0,1,0,0
1,2020-01-21,16,1,0,1,0,0
2,2020-01-22,16,4,3,1,0,0


In [38]:
from datetime import datetime as dt
time['date'] = pd.to_datetime(time['date'])
start = dt.strptime('2020-02-17', '%Y-%m-%d')
mid = dt.strptime('2020-03-05', '%Y-%m-%d')

fig = go.Figure()
clr = [DEFAULT_PLOTLY_COLORS[3] if start <= d <= mid else DEFAULT_PLOTLY_COLORS[7] for d in time['date']]
fig.add_trace(go.Bar(x=time['date'],y=time['confirmed'].diff(), name='confirmed', marker=dict(color=clr)))

fig.update_layout(title='<b>일단위 확진자 수</b>', **layout_setting)

fig.show()

# Stacked Bar 그래프 : 누적에 색깔별로 구분을 주기

In [19]:
timeAge.head(3)

,date,time,age,confirmed,deceased
0,2020-03-02,0,0s,32,0
1,2020-03-02,0,10s,169,0
2,2020-03-02,0,20s,1235,0


In [39]:
fig = px.bar(timeAge, x='date', y='confirmed', hover_data=['age'], color='age', )

fig.update_layout(title='<b>연령별 확진자 추이</b>', **layout_setting)
fig.show()

# Scatter Plot + Line Plot

In [40]:
timeAge_Date = timeAge.pivot_table(index = ['date'], columns=['age'], aggfunc=sum)
timeAge_Date['deceased']

fig = go.Figure()
for col in timeAge_Date['deceased'].columns:
    fig.add_trace(go.Scatter(x=timeAge_Date.index,y= timeAge_Date['deceased'][col],
                 mode='lines+markers', name=col))
# ... #
fig.update_layout(title='<b>연령별 사망자 추이</b>', **layout_setting)

fig.show()


# Plotly : Subplot 적용하는 법

In [50]:
Gender_Date = timeGender.pivot_table(index = ['date'],columns=['sex'], aggfunc=sum)
Gender_Date

confirmed       deceased        time     
sex           female  male   female male female male
date                                                
2020-03-02      2621  1591        9   13      0    0
2020-03-03      3002  1810       12   16      0    0
2020-03-04      3332  1996       12   20      0    0
2020-03-05      3617  2149       14   21      0    0
2020-03-06      3939  2345       17   25      0    0
...              ...   ...      ...  ...    ...  ...
2020-06-26      7218  5384      131  151      0    0
2020-06-27      7241  5412      131  151      0    0
2020-06-28      7265  5450      131  151      0    0
2020-06-29      7287  5470      131  151      0    0
2020-06-30      7305  5495      131  151      0    0

[121 rows x 6 columns]

In [51]:
Gender_Confirmed = Gender_Date['confirmed']
Gender_Deceased = Gender_Date['deceased']

print(Gender_Confirmed)
print(Gender_Deceased)

sex         female  male
date                    
2020-03-02    2621  1591
2020-03-03    3002  1810
2020-03-04    3332  1996
2020-03-05    3617  2149
2020-03-06    3939  2345
...            ...   ...
2020-06-26    7218  5384
2020-06-27    7241  5412
2020-06-28    7265  5450
2020-06-29    7287  5470
2020-06-30    7305  5495

[121 rows x 2 columns]
sex         female  male
date                    
2020-03-02       9    13
2020-03-03      12    16
2020-03-04      12    20
2020-03-05      14    21
2020-03-06      17    25
...            ...   ...
2020-06-26     131   151
2020-06-27     131   151
2020-06-28     131   151
2020-06-29     131   151
2020-06-30     131   151

[121 rows x 2 columns]


In [47]:
fig = make_subplots(rows=1, cols=2, horizontal_spacing= 0.15, subplot_titles=('<b>남여 확진자 수</b>','<b>남여 사망자 수</b>'))

fig.add_trace(go.Scatter(x=Gender_Confirmed.index, y=Gender_Confirmed['male'], mode='lines', name="Male", line=dict(color='#3370ff')), row=1, col=1)
fig.add_trace(go.Scatter(x=Gender_Confirmed.index, y=Gender_Confirmed['female'], mode='lines', name="Female", line=dict(color='#ff0d5f')), row=1, col=1)
fig.add_trace(go.Scatter(x=Gender_Deceased.index, y=Gender_Deceased['male'], mode='lines', name="Male", showlegend=False, line=dict(color='#3370ff')), row=1, col=2)
fig.add_trace(go.Scatter(x=Gender_Deceased.index, y=Gender_Deceased['female'], mode='lines', name="Female", showlegend=False, line=dict(color='#ff0d5f')), row=1, col=2)

fig.update_layout(title='<b>성별 확진자 및 사망자 수</b>', font = layout_setting['font'], showlegend=True)

fig.update_xaxes(title_text="월", row=1, col=1)
fig.update_xaxes(title_text="월", row=1, col=2)
fig.update_yaxes(title_text="명수", row=1, col=1)
fig.update_yaxes(title_text="명수", row=1, col=2)

for i in fig['layout']['annotations']:
    i['font'] = dict(size=18)

fig.show()

# subplot 만들기

In [48]:
Province_Date = timeProvince.pivot_table(index='date',columns='province')
top = Province_Date['confirmed'].sum().sort_values(ascending=False)[:5]

fig = make_subplots(rows=2, cols=1, subplot_titles=('<b>지역별 확진자 추이</b>','<b>지역별 사망자 추이</b>'))

for i, col in enumerate(list(top.index)):
    fig.add_trace(go.Scatter(x=Province_Date.index[20:],
                             y= Province_Date['confirmed'].iloc[20:,:][col],
                             mode='lines', name=col,
                            line=dict(color=DEFAULT_PLOTLY_COLORS[i], width=2)),row=1, col=1)

for i, col in enumerate(list(top.index)):
    fig.add_trace(go.Scatter(x=Province_Date.index[20:],
                             y= Province_Date['deceased'].iloc[20:,:][col],
                             mode='lines', name=col, showlegend=False,
                             line=dict(color=DEFAULT_PLOTLY_COLORS[i], width=2)), row=2, col=1)


fig.update_xaxes(title_text="Month", row=1, col=1)
fig.update_xaxes(title_text="Month", row=2, col=1)
fig.update_yaxes(title_text="Number", row=1, col=1)
fig.update_yaxes(title_text="Number", row=2, col=1)

fig.update_layout(title='<b>확진자 상위 7개 지역의 추이</b>', height=800,
                  font=dict(size=18,color='#60606e',family='Franklin Gothic'))
for i in fig['layout']['annotations']:
    i['font'] = dict(size=18)
fig.show()

# Subplot 만들기 + legend위치 지정

In [49]:
table = Province_Date
provinces = timeProvince['province'].unique()
isolated = pd.DataFrame(table['confirmed'] - table['deceased'] - table['released'])

fig = make_subplots(rows=6, cols=3, horizontal_spacing=0.03,
                    vertical_spacing= 0.05,
                   subplot_titles=([f'<b>{prov}</b>' for prov in provinces]))

for i, province in enumerate(provinces):
    row, col, legend = i//3 + 1, i%3 + 1, False
    if i == len(provinces)-1 :
        legend = True
    fig.add_trace(go.Scatter(x=table.index, y=table['confirmed'][province],
                     mode='lines', name="Confirmed", 
                     marker=dict(color = DEFAULT_PLOTLY_COLORS[3]), showlegend=legend), row=row, col=col)
    fig.add_trace(go.Scatter(x=table.index, y=table['deceased'][province],
                     mode='lines', name="Deceased", 
                     marker=dict(color = DEFAULT_PLOTLY_COLORS[2]), showlegend=legend), row=row, col=col)
    fig.add_trace(go.Scatter(x=table.index, y=table['released'][province],
                     mode='lines', name="Released",
                     marker=dict(color = DEFAULT_PLOTLY_COLORS[9]), showlegend=legend), row=row, col=col)
    fig.add_trace(go.Scatter(x=table.index, y=isolated[province],
                     mode='lines', name="Isolated",
                     marker=dict(color = DEFAULT_PLOTLY_COLORS[1]), showlegend=legend), row=row, col=col)

fig.update_layout(title='<b>지역별 환자 추이</b>',
                  height = 2000, font = layout_setting['font'],
                  legend=dict(x=0.7, y=0.05, traceorder="normal",
                             font=dict(family="sans-serif", size=18)))
for i in fig['layout']['annotations']:
    i['font'] = dict(size=20)

fig.show()